In [1]:
import rom_inverse as ri
import torch
import matplotlib.pyplot as plt
# import sklearn.gaussian_process.kernels as kernels
from poisson_fem import PoissonFEM
import ROM
import numpy as np
import scipy as sp
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import os
import pyro.contrib.gp as gp
smoke_test = ('CI' in os.environ)  # ignore; used to check code integrity in the Pyro repo
assert pyro.__version__.startswith('1.2.1')
pyro.enable_validation(True)       # can help with debugging
import time

In [ ]:
lin_dim_fom = 32                      # Linear number of rom elements

kernel = gp.kernels.RBF(input_dim=2, variance=torch.tensor(2.), lengthscale=torch.tensor([.3, .1]))
permeability_random_field = ri.DiscretizedRandomField(kernel=kernel, discretization_vector=
                            (torch.linspace(1/(2*lin_dim_fom), (lin_dim_fom - .5)/lin_dim_fom, lin_dim_fom),
                             torch.linspace(1/(2*lin_dim_fom), (lin_dim_fom - .5)/lin_dim_fom, lin_dim_fom)),
                                                     nugget=1e-3)

In [ ]:
permeability_random_field.plot_realizations()

In [ ]:
a = torch.tensor([1, 1, 0])               # Boundary condition function coefficients


# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(torch.ones(lin_dim_fom)/lin_dim_fom)
# mesh.plot()

def origin(x):
    return torch.abs(x[0]) < torch.finfo(torch.float32).eps and torch.abs(x[1]) < torch.finfo(torch.float32).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return torch.abs(x[0]) < torch.finfo(torch.float32).eps or torch.abs(x[1]) < torch.finfo(torch.float32).eps or \
            torch.abs(x[0]) > 1.0 - torch.finfo(torch.float32).eps or torch.abs(x[1]) > 1.0 - torch.finfo(torch.float32).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [ ]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [ ]:
# define fom
fom = ROM.ROM(mesh, K, rhs, lin_dim_fom**2)
# Change for non unit square domains!!
xx, yy = torch.meshgrid((torch.linspace(0, 1, lin_dim_fom), torch.linspace(0, 1, lin_dim_fom)))
X = torch.cat((xx.flatten().unsqueeze(1), yy.flatten().unsqueeze(1)), 1)
fom.mesh.get_interpolation_matrix(X)

In [ ]:
fom_autograd = fom.get_autograd_fun()

In [ ]:
lmbda = permeability_random_field.sample_permeability(n_samples=3)

In [ ]:
img = plt.imshow(fom_autograd(lmbda[0, :]).view(lin_dim_fom, lin_dim_fom))
plt.colorbar(img)

In [ ]:
lmbda = torch.rand(lin_dim_fom**2, requires_grad=True)
x = fom_autograd(lmbda)
loss = torch.norm(x)
loss.backward()

In [ ]:
def lossfun(x):
    return torch.norm(fom_autograd(x)) 

In [ ]:
# from torch.autograd import gradcheck
# lmbda = torch.randn(lin_dim_fom**2, dtype=torch.double, requires_grad=True)
# test = gradcheck(lossfun, lmbda, eps=1e-3, atol=1e-4)
# print(test)

In [ ]:
# define pyro model
beta = 1.0  # inverse temperature of observations
def uncertainty_propagation():
    lambda_f = permeability_random_field.sample()
    uf = fom_autograd(lambda_f)
    uf_observed = pyro.sample('uf_observed', dist.Normal(uf, torch.ones_like(uf)/beta))
    return uf_observed   

In [ ]:
# nuts_kernel = NUTS(uncertainty_propagation)
# mcmc = MCMC(nuts_kernel, num_samples=100, warmup_steps=100, num_chains=1)
# mcmc.run()
# mcmc.summary()

In [ ]:
beta = 20
scale_tril = torch.cholesky(permeability_random_field.get_covariance_matrix())
mu_zero = torch.zeros(permeability_random_field.X.shape[0])
def joint_posterior():
    x = pyro.sample('x', dist.MultivariateNormal(mu_zero, scale_tril=scale_tril))
    lambdaf = torch.exp(x)
    uf = fom_autograd(lambdaf)
    uf_observed = pyro.sample('uf_observed', dist.Normal(uf, torch.ones_like(uf)/beta))
    return uf_observed

In [ ]:
# nuts_kernel = NUTS(joint_posterior)
# mcmc = MCMC(nuts_kernel, num_samples=100, warmup_steps=100, num_chains=1)
# mcmc.run()
# mcmc.summary()

In [ ]:
print('x == ', x := permeability_random_field.sample_log_permeability())
print('uf_observed == ', uf_observed := fom_autograd(torch.exp(x)))

In [ ]:
def conditioned_posterior(uf_observed):
    return pyro.condition(joint_posterior, data={"uf_observed": uf_observed})

In [ ]:
nuts_kernel = NUTS(conditioned_posterior(uf_observed))
mcmc = MCMC(nuts_kernel, num_samples=200, warmup_steps=100, num_chains=1)
mcmc.run()
mcmc.summary()

In [ ]:
plt.plot(x[0])
plt.plot(torch.mean(mcmc.get_samples()['x'], 0))

In [ ]:
fig = plt.figure(figsize=(15, 7))
ax = plt.subplot(1, 2, 1)
im0 = plt.imshow(x[0].view(lin_dim_fom, lin_dim_fom))
plt.colorbar(im0)
ax = plt.subplot(1, 2, 2)
im1 = plt.imshow(torch.mean(mcmc.get_samples()['x'], 0).view(lin_dim_fom, lin_dim_fom))
plt.colorbar(im1)

In [ ]:
torch.cholesky(permeability_random_field.get_covariance_matrix())

In [ ]:
plt.imshow(permeability_random_field.get_covariance_matrix().detach().numpy())

In [ ]:
torch.det(permeability_random_field.get_covariance_matrix())

In [ ]:
permeability_random_field.get_covariance_matrix()[permeability_random_field.get_covariance_matrix() < 1e-10] = .0

In [ ]:
torch.det(permeability_random_field.get_covariance_matrix() + 1e-5*torch.eye(81))

In [ ]:
permeability_random_field.get_covariance_matrix().min()

In [ ]:
permeability_random_field.get_covariance_matrix().size()

In [ ]:
permeability_random_field.X.shape

In [ ]:
scale_tril = torch.cholesky(cov)

In [ ]:
scale_tril

In [ ]:
torch.save(x, 'x_true.pt')

In [ ]:
uf_observed

In [ ]:
mcmc.get_samples()['x']

In [2]:
x = torch.rand(16)

In [3]:
torch.me

tensor([0.4143, 0.7055, 0.9414, 0.9029, 0.3258, 0.0392, 0.3680, 0.0745, 0.6988,
        0.6052, 0.1571, 0.3206, 0.9483, 0.3301, 0.6511, 0.7495])